<div dir="rtl" class="alert alert-block alert-info"> <font size="5" face="HM XNiloofar"> 
<b> یادگیری تقویتی - دکتر گلستانی: تمرین شبیه‌سازی سوم </b>
</font> </div>

<div dir="rtl"> <font size="4" face="HM XNiloofar"> 
<br />
مهلت تحویل تمرین: ۱۸ مرداد ۱۴۰۱ <br />
فقط قسمت‌های <code>TODO</code> را پر کنید و  Jupyter Notebook تکمیل شده را به فرمت <code>ipynb</code> و <code>html</code> به عنوان گزارش خود در CW آپلود کنید. <br />
ابهامات و سوالات خود در مورد این تمرین را می توانید با طراح تمرین مطرح کنید. <br />
<div dir="ltr">@hamidreza_ehteram</div> <br />  
در این تمرین شما در بازی معروف Breakout در Atari الگوریتم‌ DQN + Experience Replay + Target Network  را پیاده‌سازی می‌کنید. <br />
<br />
توجه داشته باشید که آموزش Agent در این بازی حداقل ۱۰ ساعت زمان می‌برد. پس از الان شروع کنید!
<br />
کدهای این Notebook در محیط 
   <a href="https://colab.research.google.com/">Google Colab</a>
    بدون نصب کتاب‌خانه‌ای قابل اجراست.
    <br />
    در این تمرین برای آموزش شبکه‌های عصبی از کتاب‌خانه‌ی 
    <a href="https://pytorch.org/">Pytorch</a>
    استفاده می‌کنیم (چنانچه با این محیط آشنایی ندارید از 
    <a href="https://pytorch.org/tutorials/">این لینک</a>
    استفاده کنید). 
    <br />
    توصیه می‌شود برای آموزش شبکه‌های عصبی از GPU استفاده کنید.
 </font> </div>

***You can jump directly to these sections:***

+ [A. Introduction](#A.-Introduction)
    + [1. Algorithm](#1.-Algorithm)
    + [2. Game environment](#2.-Game-environment)
    + [3. Q-network architecture](#3.-Q-network-architecture)
    + [4. Deepmind helper functions](#4.-Deepmind-helper-functions)
+ [B. An experience with the environment](#B.-An-experience-with-the-environment)
    + [1. Start game](#1.-Start-game)
    + [2. end-of-life vs end-of-episode](#2.-end-of-life-vs-end-of-episode)
    + [3. Reward](#3.-Reward)
+ [C. Training](#C.-Training)
    + [1. Network](#1.-Network)
    + [2. Hyperparameters](#2.-Hyperparameters)
    + [3. Learning](#3.-Learning)
    + [4. Result](#4.-Result)
+ [Give Us Feedback](#Give-Us-Feedback)
---

<div dir="rtl" style="margin: 20px" class="alert alert-block alert-success"> <font size="4" face="HM XNiloofar"> <b>  
    <a href="https://en.wikipedia.org/wiki/Breakout_(video_game)">Atari Breakout Game:</a> </b>
در این تمرین قرار است شما یک Agent را آموزش دهید که امتیاز خود در بازی را حداکثر کند. محیط با حذف هر خانه‌ی رنگی یک امتیاز به Agent می‌دهد. توجه داشته باشید که Agent فقط اسکرین بازی را مشاهده می‌کند (دقیقا مثل یک انسان که Atari 🎮 بازی می‌کند) و با هیچ کانسپتی آشنا نیست. یعنی نمی‌داند توپ چیست یا این که اکشن‌های بازی دقیقا چه کاری انجام می‌دهند. جالب اینجاست درحالی که انسان با این مفاهیم کاملا آشناست ولی Agent می‌تواند به عملکردی بهتر از انسان دست یابد (
    <a href="https://www.youtube.com/watch?v=V1eYniJ0Rnk">این لینک</a>
    را ببینید).
</font> </div>

---

# A. Introduction

## 1. Algorithm

<div dir="rtl"> <font size="4" face="HM XNiloofar">    
    به یاد بیاورید که در کلاس درس شما با الگوریتم Q-Learning و SARSA در بحث model-free آشنا شدید. سپس الگوریتم SARSA را با روش Approximation فرا گرفتید. الگوریتم DQN ورژن Approximation الگوریتم Q-Learning است. عملکرد این الگوریتم  با دو تکنیک Experience Replay و Target Network به طور قابل ملاحضه‌ای بهبود می‌یابد.
<br />
در تکنیک Experience Replay به جای این که پارامترهای شبکه از روی آخرین ترنزیشن، یعنی چهارتایی 
<div dir="ltr"> <center> <b> <code>transition = [state, action, reward, next_state, done]</code> </b> </center></div> 
 به‌روز شوند، به کمک یک batch تصادفی از ترنزیشن‌های گذشته به‌روز‌رسانی صورت می‌گیرد. ترنزیشن‌های گذشته در یک بافر 
 <i>D</i>
 با سایز محدود ذخیره می‌گردد.
<br />
در تکنیک Target Network مقدار تارگت زوج استیت و اکشن در تابع هزینه از روی شبکه Target یعنی
 Q(.,.;&Theta;')
که پارامتر‌هایی متفاوت از شبکه Learner یعنی 
Q(.,.;&Theta;)
دارد محاسبه می‌شود. پارامترهای شبکه Target به طور متناوب برای k گام بدون تغییر می‌ماند و سپس برابر با پارامتر‌های شبکه Learner   می‌شود؛
&Theta;' = &Theta;.
<br />
تابع هزینه به صورت زیر در می‌آید:
</font> </div> 

$$L(\theta) = \mathbb{E}_{(S_t, A_t, R_{t+1}, S_{t+1})\sim \mathcal{D}} \bigg[\Big(R_{t+1} + \gamma \max_{a' \in \mathcal{A}} Q(S_{t+1},a';\theta') - Q(S_{t},A_t;\theta)\Big)^2\bigg]$$

<div dir="rtl"> <font size="4" face="HM XNiloofar"> 
    توجه داشته باشید که اگر
   <code>next_state</code> یک استیت نهایی باشد، مقدار تارگت برابر <code>reward</code> خواهد بود.  
    <br />
    <br />
برای پیاده‌سازی الگوریتم اسلاید ۱ الی ۳۷ 
    <a href="http://ce.sharif.edu/courses/98-99/2/ce719-1/resources/root/Slides/Deep%20RL.pdf">این منبع</a>
    را مطالعه کنید. الگوریتمی که در این تمرین پیاده‌سازی می‌کنید در <b>اسلاید ۲۵</b> آورده شده است. 
    پارامترها و ساختار شبکه در این تمرین منطبق بر
    <a href="https://storage.googleapis.com/deepmind-data/assets/papers/DeepMindNature14236Paper.pdf">DeepMind's Nature paper</a>
    است.
    <br />
    برای اطلاعات بیشتر مطالعه‌ی مقاله‌ی زیر توصیه می‌گردد:
</font> </div> 
Volodymyr Mnih, Koray Kavukcuoglu, David Silver, Andrei A Rusu, Joel Veness, Marc G Bellemare, Alex Graves, Martin Riedmiller, Andreas K Fidjeland, Georg Ostrovski, et al. Human-level control through deep reinforcement learning. nature, 518(7540):529–533, 2015. 5



<div dir="rtl"> <font size="4" face="HM XNiloofar">
    <br />
    <br />
پس از مطاله‌ی منابع فوق به سوالات زیر پاسخ دهید:
<br />
۱. تفاوت الگوریتم DQN و SARSA را بیان کنید.
<br />
۲.  مزایای استفاده از Experience Replay چیست؟ افزایش اندازه‌ی بافر چه تاثیر مثبت و منفی بر الگوریتم دارد؟
<br />
۳. مزایای استفاده از Target Network چیست؟ افزایش دوره تناوب به‌روز‌رسانی پارامترهای این شبکه (k) چه تاثیری بر الگوریتم دارد؟
<br />
۴. گرادیان تابع هزینه نسبت به پارامتر‌های شبکه Learner را محاسبه کنید. توجه داشته که گرادیان را نسبت به پارامترهای شبکه Target محاسبه نمی‌کنیم.
</font> </div> 


**Your answer: ```TODO ```**

## 2. Game environment

<div dir="rtl"> <font size="4" face="HM XNiloofar">
<a href="https://www.gymlibrary.ml/ ">کتاب خانه Gym</a> 
    یکی از منابع مفید برای استفاده از محیط‌های آماده برای ارزیابی الگوریتم‌های RL است. کد زیر طریق استفاده از یکی از بازی‌های این کتاب‌خانه را نشان می‌دهد: 
</font> </div> 

```python
# libraries
import gym
! wget http://www.atarimania.com/roms/Roms.rar
! mkdir /content/ROM/
! unrar e /content/Roms.rar /content/ROM/
! python -m atari_py.import_roms /content/ROM/
```

```python
# General Code
env = gym.make('CartPole-v0')
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()
```
<div dir="rtl"> <font size="4" face="HM XNiloofar">
 با مطالعه‌ی داکیومنت‌های سایت Gym بیان کنید هر خط کد بالا چه کاری انجام می دهد و متغیرهای خروجی در انجام هر اکشن نشان دهنده‌ی چیست؟
 </font> </div> 

**Your answer: ```TODO ```**

## 3. Q-network architecture


<div dir="rtl"> <font size="4" face="HM XNiloofar">
 محیط ما در این تمرین بازی Breakout است. در این بازی ۴ اکشن <code>('NOOP', 'FIRE', 'RIGHT', 'LEFT')</code> داریم. با انجام هر اکشن، محیط به ما ۴ متغیر <code>observation, reward, done, info</code> را می دهد. <code>observation</code> اسکرین بازی را نشان می‌دهد که ابعاد <code>(3, 160, 210)</code> دارد که عدد 3 بیانگر RGB بودن صفحه است. (در این تمرین از متغیر  <code>info</code> استفاده نمی‌کنیم). 
<br />
 معماری شبکه‌ی عصبی بازی در 
<a href="http://ce.sharif.edu/courses/98-99/2/ce719-1/resources/root/Slides/Deep%20RL.pdf">اسلاید ۹</a> 
     نشان داده شده است. این ساختاری است که مقاله DQN برای آموزش بازی‌ها از آن استفاده کرده. 
    <br />
    <br />
    حال به سوالات زیر پاسخ دهید:
 <br />   
۱.    
 ساختار تابع Q(s,a;&Theta;) به چه صورت طراحی شده است؟ 
 <br />  
۲. 
    ورودی شبکه‌ی عصبی در این ساختار فقط استیت بازی است درحالی که انتظار داریم زوج استیت و اکشن باشد. نشان دهید ساختار فوق یک تابع معادل دارد که زوج استیت و اکشن را دریافت و value را در خروجی می‌دهد.
 <br />  
۳.    
    استفاده از ساختار پیشنهاد شده در این اسلاید چه مزیتی بر یک ساختار شبکه عصبی دارد که زوج استیت و اکشن را دریافت و با ترکیب آن‌ها به کمک اجرای چند لایه شبکه عصبی value را در خروجی می‌دهد؟ 
 <br />  
۴.    
 ورودی شبکه‌ی عصبی به جای این که اسکرین یک لحظه‌ی بازی باشد، اسکرین ۴ لحظه‌ی متوالی در بازی است. چرا؟ افزایش این عدد چه تاثیری بر عملکرد الگوریتم خواهد داشت؟
 <br />  
۵.
    سایز ورودی شبکه 
    <code>(4, 84, 84)</code>
    و سایز خروجی
    <code>4</code>
    است. این اعداد نشان دهنده‌ی چه هستند؟
 </font> </div> 

**Your answer: ```TODO ```**

## 4. Deepmind helper functions

<div dir="rtl"> <font size="4" face="HM XNiloofar">
    در این تمرین ما از دو تابع کمکی که توسط Deepmind آماده شده است استفاده می‌کنیم. source code این دو تابع در ادامه نوشته شده است. آن را مطالعه و سپس به پرسش پاسخ دهید.
 </font> </div> 
 
Deepmind functions (for Atari games)

```python
# to use stable_baselines.common.atari_wrappers
%tensorflow_version 1.x
!pip install stable-baselines[mpi]==2.10.2

from stable_baselines.common.atari_wrappers import make_atari, wrap_deepmind
```
```python
# Use the Baseline Atari environment because of Deepmind helper functions
env = make_atari('BreakoutNoFrameskip-v4')
# Warp the frames, grey scale, stake four frame and scale to smaller ratio
env = wrap_deepmind(env, frame_stack=True, scale=True)
```
It preprocesses the environment:

Source: 

https://github.com/openai/baselines/blob/ea25b9e8b234e6ee1bca43083f8f3cf974143998/baselines/common/atari_wrappers.py

functions:

```python
def make_atari(env_id, max_episode_steps=None):
    env = gym.make(env_id)
    assert 'NoFrameskip' in env.spec.id

    # Sample initial states by taking random number of no-ops on reset.
    env = NoopResetEnv(env, noop_max=30)

    # Return only every `skip`-th frame
    # Repeat action (for skip` times), sum reward (in these repeats), and max over last observations.
    env = MaxAndSkipEnv(env, skip=4)

    if max_episode_steps is not None:
        env = TimeLimit(env, max_episode_steps=max_episode_steps)
    return env

def wrap_deepmind(env, episode_life=True, clip_rewards=True, frame_stack=False, scale=False):
    """Configure environment for DeepMind-style Atari.
    """
    if episode_life:

        # Make end-of-life == end-of-episode, but only reset on true game over.
        env = EpisodicLifeEnv(env)

    if 'FIRE' in env.unwrapped.get_action_meanings():

        # Take action on reset for environments that are fixed until firing.
        env = FireResetEnv(env)

    # Warp frames to 84x84 as done in the Nature paper and later work. (and convert to gray scale)
    env = WarpFrame(env)

    if scale:

        # careful! This undoes the memory optimization, use with smaller replay buffers only.
        # code: np.array(observation).astype(np.float32) / 255.0 ----> range [0,1]
        env = ScaledFloatFrame(env)

    if clip_rewards:

        # Bin reward to {+1, 0, -1} by its sign.
        env = ClipRewardEnv(env)

    if frame_stack:

        # Stack k last frames. Returns lazy array, which is much more memory efficient.
        # observation ---> state 
        # All the information we need to choose an action
        # This will allow the agent to take the velocity of things into account from one image.
        # The direction of agent's movement in the previous several frames
        env = FrameStack(env, 4)

    return env
```

Also, you can see the following link (which implements some similar functions):

https://pytorch.org/tutorials/intermediate/mario_rl_tutorial.html#preprocess-environment


<div dir="rtl"> <font size="4" face="HM XNiloofar">
    <br />
    <br />
با مطالعه‌ی کدهای بالا توضیح دهید تابع <code>make_atari</code> چه تفاوتی با <code>gym.make</code> دارد؟ تابع <code>wrap_deepmind</code> چه پیش‌پردازش‌هایی را بر روی محیط انجام می‌دهد؟ 
 </font> </div>

**Your answer: ```TODO ```**

---

# B. An experience with the environment

<div dir="rtl"> <font size="4" face="HM XNiloofar"> 
    در این قسمت می‌خواهیم با محیط بازی و توابع آن آشنا شویم.
</font> </div>

<div dir="rtl"> <font size="4" face="HM XNiloofar"> 
Cell های زیر به منظور آشنایی شما با محیط بازی نوشته شده‌اند. آن‌ها را اجرا کنید.
</font> </div>

### libraries

In [ ]:
# libraries
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import datetime
import time
import itertools

In [ ]:
# libraries
import gym

! wget http://www.atarimania.com/roms/Roms.rar
! mkdir /content/ROM/
! unrar e /content/Roms.rar /content/ROM/
! python -m atari_py.import_roms /content/ROM/

In [ ]:
# libraries

# to use stable_baselines.common.atari_wrappers
%tensorflow_version 1.x
!pip install stable-baselines[mpi]==2.10.2

from stable_baselines.common.atari_wrappers import make_atari, wrap_deepmind

### Gif function

In [ ]:
# visualize game by gif

from matplotlib import animation

def save_frames_as_gif(frames, filename=None):
    """
    Save a list of frames as a gif
    """
    patch = plt.imshow(frames[0])
    plt.axis('off')
    def animate(i):
        patch.set_data(frames[i])
    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    if filename:
        anim.save(filename, dpi=72, writer='pillow')


## 1. Start game

<div dir="rtl"> <font size="4" face="HM XNiloofar"> 
در زیر به سه روش شروع به بازی در محیط می‌کنیم. به خروجی‌های Cell ها و gif های خروجی توجه کنید و مختصر درباره عملکرد سه روش توضیح دهید. 
</font> </div>

### Original function

Using `env = gym.make("BreakoutNoFrameskip-v4")`

In [ ]:
env = gym.make("BreakoutNoFrameskip-v4")

observation = env.reset()

print(observation.dtype, observation.shape)
plt.imshow(observation)
print(env.action_space)

In [ ]:
frames = []
rewards = []

for i_episode in range(1):
    observation = env.reset()
    frames.append(observation)
    for t in range(100000):
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        frames.append(observation)
        rewards.append(reward)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

save_frames_as_gif(frames, filename='randompolicy1.gif')
print(sum(rewards), rewards)

### Deepmind function

Using `env = make_atari("BreakoutNoFrameskip-v4")`

In [ ]:
# Use the Baseline Atari environment because of Deepmind helper functions
env = make_atari("BreakoutNoFrameskip-v4")

observation = env.reset()

print(observation.dtype, observation.shape)
plt.imshow(observation)
print(env.action_space)

In [ ]:
frames = []
rewards = []

for i_episode in range(1):
    observation = env.reset()
    frames.append(observation)
    for t in range(100000):
        # env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        frames.append(observation)
        rewards.append(reward)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

save_frames_as_gif(frames, filename='randompolicy2.gif')
print(sum(rewards), rewards)

### Deepmind function + preprocessing

Using `env = make_atari("BreakoutNoFrameskip-v4")` and `env = wrap_deepmind(env, frame_stack=True, scale=True)`

In [ ]:
# Use the Baseline Atari environment because of Deepmind helper functions
env = make_atari("BreakoutNoFrameskip-v4")

# Warp the frames, grey scale, stake four frame and scale to smaller ratio
env = wrap_deepmind(env, frame_stack=True, scale=True)

observation = env.reset()
for i in range(10):
  action = env.action_space.sample()
  observation, reward, done, info = env.step(action)
observation = np.array(observation)

print(observation.dtype, observation.shape)
plt.figure()
plt.imshow(observation[:,:,0])
plt.figure()
plt.imshow(observation[:,:,1])
plt.figure()
plt.imshow(observation[:,:,2])
plt.figure()
plt.imshow(observation[:,:,3])
print(env.action_space)

In [ ]:
frames = []
rewards = []

for i_episode in range(1):
    observation = env.reset()
    observation = np.array(observation)
    frames.append(observation)
    for t in range(100000):
        # env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        observation = np.array(observation)
        frames.append(observation)
        rewards.append(reward)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

save_frames_as_gif(frames, filename='randompolicy3.gif')
print(sum(rewards), rewards)

**Your answer: ```TODO ```**

## 2. end-of-life vs end-of-episode

<div dir="rtl"> <font size="4" face="HM XNiloofar"> 
تابع <code>EpisodicLifeEnv</code> در تابع <code>wrap_deepmind</code> باعث چه تغییراتی در محیط شده؟ 
</font> </div>

**Your answer: ```TODO ```**

## 3. Reward

<div dir="rtl"> <font size="4" face="HM XNiloofar"> 
چه زمانی محیط به Agent ریوارد می‌دهد؟ 
</font> </div>

**Your answer: ```TODO ```**

---

# C. Training

## Mount - Libraries - Functions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# libraries
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision
import torch.backends.cudnn as cudnn

import os
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

import datetime
import time
import itertools

In [ ]:
# save and load
save_path = F'/content/drive/MyDrive/DQN'
os.makedirs(save_path, exist_ok=True)

# save function
def mysave(state, name):
  from google.colab import drive
  drive.mount('/content/drive')
  path = os.path.join(save_path, name+'.pth')
  print('Saving..')
  torch.save(state, path)

# test
# save
mysave(state='no', name='Test01')
# load
state = torch.load('/content/drive/MyDrive/DQN/Test01.pth')
print(state)

In [ ]:
# check device (USE CUDA!)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print ('Your system: ' + str(device))

## 1. Network

<div dir="rtl"> <font size="4" face="HM XNiloofar"> 
Cell زیر مربوط به پیاده‌سازی معماری شبکه‌ عصبی (۳ لایه Convolutional و ۲ لایه Fully Connected) است. آن را منطبق بر معماری شبکه‌ی عصبی در 
<a href="https://storage.googleapis.com/deepmind-data/assets/papers/DeepMindNature14236Paper.pdf">DeepMind's Nature paper</a>
    تکمیل کنید (معماری مدل در بخش Methods section یافت می‌شود).
<br /> 
 سپس Cell مربوط به ساخت مدل‌های شبکه Target و Learner را اجرا کنید.
توضیح دهید هر خط این Cell  چه کاری انجام می‌دهد. رابطه ریاضی <code>nn.SmoothL1Loss</code> را بنویسید و مزیت استفاده از آن به جای mean squared error را ذکر کنید.  
</font> </div>

In [ ]:
class Network(nn.Module):
    
    def __init__(self, num_actions):
        super(Network, self).__init__()
        
        # TODO

    def forward(self, x):
        
        # TODO
        
        return out


In [ ]:
action_space = [0, 1, 2, 3] # ['NOOP', 'FIRE', 'RIGHT', 'LEFT']

In [ ]:
# build model_learner
print('==> Building learner model..')
model_learner = Network(len(action_space)).to(device)
if device.type == 'cuda':
    model_learner = torch.nn.DataParallel(model_learner)
    cudnn.benchmark = True

# build model_target
print('==> Building target model..')
model_target = Network(len(action_space)).to(device)
if device.type == 'cuda':
    model_target = torch.nn.DataParallel(model_target)
    cudnn.benchmark = True

# model_target parameters are frozen.
for p in model_target.parameters():
    p.requires_grad = False

# update model_target
model_target.load_state_dict(model_learner.state_dict())

# loss function
criterion = nn.SmoothL1Loss().to(device)

**Your answer: ```TODO ```**

## 2. Hyperparameters

<div dir="rtl"> <font size="4" face="HM XNiloofar"> 
پارامترهای پیشنهادی زیر (تا حد خوبی) منطبق بر 
<a href="https://storage.googleapis.com/deepmind-data/assets/papers/DeepMindNature14236Paper.pdf">DeepMind's Nature paper</a>
انتخاب شده‌اند (جدول مربوطه در مقاله را ببینید). هر یک از پارامترها چه چیزی را نشان می‌دهد؟
</font> </div>

In [ ]:
# Use the Baseline Atari environment because of Deepmind helper functions
env = make_atari("BreakoutNoFrameskip-v4")

# Warp the frames, grey scale, stake four frame and scale to smaller ratio
env = wrap_deepmind(env, frame_stack=True, scale=True)

# for the loops
max_num_episode = 1000000
max_step_per_episode = 10000

# i_episode
i_episode = 0

# i_step
i_step = 0

# How often to update model_target (in steps)
update_model_target = 5000 # or 10000

# How often to train model_learner (in steps)
train_model_learner = 4

# epsilon parameters
eps = 1.0
eps_max = 1.0
eps_min = 0.1
eps_num_random_action = 50000 #(in steps)
eps_num_explor_exploit = 1000000.0 #(in steps)
eps_interval = (eps_max - eps_min) / eps_num_explor_exploit

# memory
# memory size: Deepmind paper suggests 1000000; however, this causes memory issues
# If we don't save the tensors in device, we can set it to 200000; otherwise, it can be 100000 - 150000.
memory = []
memory_size = 150000
batch_size = 32
factor_train_batch_size = 400

# discount factor
gamma = 0.99

# learning rate
lr = 0.00025 # or decrease it from 1e-3 to 1e-4 gradually.

# optimizer
optimizer = torch.optim.Adam(model_learner.parameters(), lr=lr)

# reset track
track = {}
track['reward_sum'] = [-1.0]
track['num_step'] = [-1.0]
track['eps'] = [1.0]
track['i_episode'] = [0]
track['i_step'] = [0]

# save
state_save = {'model_learner':model_learner.state_dict(), 'model_target':model_target.state_dict(), 'track':track}
mysave(state=state_save, name='RL_DQN_'+str(i_episode))

**Your answer: ```TODO ```**

## 3. Learning

<div dir="rtl"> <font size="4" face="HM XNiloofar"> 
در زیر دو cell مشاهده می‌کنید. یکی برای لود کردن یک مدل train شده از درایو شما و یکی برای پیاده‌سازی الگوریتم. 
<br />    
    Cell مربوط به الگوریتم را تکمیل و اجرا کنید. قسمت‌های مختلف کد این الگوریتم را توضیح دهید. چه تکنیک‌هایی برای پیاده‌سازی کارامد بافر استفاده شده است؟
<br />
<br />
زمانی فرایند training را متوقف کنید که Agent عملکرد خوبی داشته باشد (معیار ما در بخش Result در ادامه تمرین ذکر شده است).    
<br />    
    در کد مربوط به الگوریتم، هر ۵۰۰ episode یک بار مدل شبکه‌ها و متغیر <code>track</code> در درایو شما ذخیره می‌گردد. چنانچه به هر دلیلی فراید training متوقف شد و خواستید مجددا آن را شروع کنید، کافیست نام آخرین فایل ذخیره شده در درایو خود را در cell زیر وارد کنید و مجدد فرایند training را شروع کنید.
</font> </div>

**Your answer: ```TODO ```**

In [ ]:
# loading
state = torch.load('/content/drive/MyDrive/DQN/RL_DQN_0.pth')
track = state['track']
model_learner.load_state_dict(state['model_learner'])
model_target.load_state_dict(state['model_target'])
eps = track['eps'][-1]
i_episode = track['i_episode'][-1]
i_step = track['i_step'][-1]

print(eps, i_episode, i_step)

In [ ]:
# main loop

while i_episode <= max_num_episode:
    i_episode += 1

    # init
    state = env.reset()
    state = torch.tensor(np.transpose((np.array(state) * 255), (2, 0, 1)), dtype=torch.uint8).to(device)
    reward_sum = 0.0

    for t in range(max_step_per_episode):
        i_step += 1

        # ----------------------------------------------------------------------------
        # choose action 
        # ----------------------------------------------------------------------------
        # update epsilon
        eps = # TODO
        # epsilon greedy
        if np.random.rand() <= eps:
            action = # TODO
        else:
            model_learner.eval()
            with torch.no_grad():
              x = state.to(torch.float32).unsqueeze(0) / 255.0
              outputs = model_learner(x)
            action = # TODO   
        
        # ----------------------------------------------------------------------------
        # do the step 
        # ----------------------------------------------------------------------------
        next_state, reward, done, info = env.step(action)
        reward_sum += reward
        if done: reward = -1

        # ----------------------------------------------------------------------------
        # store transition in memory (or buffer) with efficiency 
        # ----------------------------------------------------------------------------
        state = state
        action = torch.tensor(action).to(device)
        reward = torch.tensor(reward).to(device)
        next_state = torch.tensor(np.transpose((np.array(next_state) * 255), (2, 0, 1)), dtype=torch.uint8).to(device)
        done =  torch.tensor(done).to(device)
        # transition
        transition = [state, action, reward, next_state, done]
        # add to memory
        memory.append(transition)
        # limit memory size
        if len(memory) > memory_size: del memory[0] 

        # ----------------------------------------------------------------------------
        # go to next_state 
        # ----------------------------------------------------------------------------
        state = next_state

        # ----------------------------------------------------------------------------
        # train model_learner 
        # ----------------------------------------------------------------------------
        if (len(memory) >= batch_size * factor_train_batch_size) and (i_step % train_model_learner == 0):
            # sample from memory a batch of transitions
            batch_transitions = random.sample(memory, batch_size)
            batch_state, batch_action, batch_reward, batch_next_state, batch_done = map(lambda x: torch.stack(x, 0).to(torch.float32), zip(*batch_transitions))
            batch_state, batch_action, batch_reward, batch_next_state, batch_done = batch_state/255.0, batch_action.to(torch.long), batch_reward, batch_next_state/255.0, batch_done
            # train model_learner
            model_learner.train()
            # forward
            q_predict = # TODO
            q_target = # TODO
            # loss
            loss = criterion(q_predict, q_target.detach())
            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()        
        
        # ----------------------------------------------------------------------------
        # update model_target 
        # ----------------------------------------------------------------------------
        if i_step % update_model_target == 0:
            # TODO
            print('model_target updated!')
        
        # ----------------------------------------------------------------------------
        # if done
        # ----------------------------------------------------------------------------          
        if done:
            print(f"Episode [{i_episode}/{max_num_episode}] Step [{i_step}] | reward_sum -> {reward_sum} | num_step -> {t+1} | eps -> {eps}")
            track['reward_sum'].append(reward_sum)
            track['num_step'].append(t+1)
            track['eps'].append(eps)
            track['i_episode'].append(i_episode)
            track['i_step'].append(i_step)

            # save
            if i_episode % 500 == 0:
              state_save = {'model_learner':model_learner.state_dict(), 'model_target':model_target.state_dict(), 'track':track}
              mysave(state=state_save, name='RL_DQN_'+str(i_episode))
                      
            break

state_save = {'model_learner':model_learner.state_dict(), 'model_target':model_target.state_dict(), 'track':track}
mysave(state=state_save, name='RL_DQN_'+str(i_episode))
env.close()

## 4. Result

<div dir="rtl"> <font size="4" face="HM XNiloofar"> 
برای کسب نمره‌ی کامل باید 
     به طور متوسط در هر episode بازی ۸ امتیاز کسب کنید (بعنی میانگین ۵۰۰ المنت آخر در <code>track['reward_sum']</code> بیشتر از ۸ باشد). برای این عملکرد حدودا نیاز به اجرای ۵۰۰۰۰ episode دارید. اگر عملکرد agent شما بهتر از این عدد باشد نمره‌ی امتیازی کسب می‌کنید.
<br /> 
<br /> 
در انتهای training، آخرین فایل ذخیره شده در Google Drive را به صورت دسترسی همگانی قرار دهید و cell زیر را کامل کنید تا هر کس بتواند با اجرای cell به پارامترهای train  شده و متغیر <code>track</code> در فایل نهایی دست‌ یابد. 
<br /> 
<br /> 
سپس cell مربوط به ترسیم نتایج را اجرا و خروجی را آنالیز کنید.    
<br />
<br /> 
در انتها cell های مربوط به gif را اجرا و فایل    <code>result.gif</code> را دانلود و در درایو آپلود کرده و لینک دانلود آن را در قسمت مربوطه قرار دهید.    
</font> </div>

### Load from link

In [ ]:
# Set file's access to 'Anyone with the link'. Then paste file's id (available in the public link).

# TODO (e.g., !gdown 10bapl28NnuYANd8baMqGGmmtOfGMWwGx)


# loading

# TODO (e.g., state = torch.load('/content/RL_DQN_4500.pth'))

track = state['track']
model_learner.load_state_dict(state['model_learner'])
model_target.load_state_dict(state['model_target'])
eps = track['eps'][-1]
i_episode = track['i_episode'][-1]
i_step = track['i_step'][-1]

print(eps, i_episode, i_step)

### Curves

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

fig, ax = plt.subplots(2, 2, figsize=(10,10))
#
ax[0,0].plot(np.arange(len(track['reward_sum'])), track['reward_sum'], label=f"w.o. moving_average")
ax[0,0].set_xlabel('episode')
ax[0,0].set_ylabel('the cumulative reward per episode')
ax[0,0].legend()
#
y = moving_average(track['reward_sum'],500)
ax[0,1].plot(np.arange(len(y)), y, label=f"w. moving_average")
ax[0,1].set_xlabel('episode')
ax[0,1].set_ylabel('the cumulative reward per episode')
ax[0,1].legend()
#
ax[1,0].plot(np.arange(len(track['num_step'])), track['num_step'], label=f"w.o. moving_average")
ax[1,0].set_xlabel('episode')
ax[1,0].set_ylabel('the number of steps per episode')
ax[1,0].legend()
#
y = moving_average(track['num_step'],500)
ax[1,1].plot(np.arange(len(y)), y, label=f"w. moving_average")
ax[1,1].set_xlabel('episode')
ax[1,1].set_ylabel('the number of steps per episode')
ax[1,1].legend()
#
plt.show()

fig, ax = plt.subplots(1, 1, figsize=(5,5))
ax.plot(np.arange(len(track['eps'])), track['eps'])
ax.set_xlabel('episode')
ax.set_ylabel('epsilon')
plt.show()

**Your answer: ```TODO ```**

### Gif

In [ ]:
from stable_baselines.common.atari_wrappers import FrameStack, ScaledFloatFrame, WarpFrame, NoopResetEnv, ClipRewardEnv, EpisodicLifeEnv, FireResetEnv, MaxAndSkipEnv, MaxAndSkipEnv

In [ ]:
class TimeLimit(gym.Wrapper):
    def __init__(self, env, max_episode_steps=None):
        super(TimeLimit, self).__init__(env)
        self._max_episode_steps = max_episode_steps
        self._elapsed_steps = 0

    def step(self, ac):
        observation, reward, done, info = self.env.step(ac)
        self._elapsed_steps += 1
        if self._elapsed_steps >= self._max_episode_steps:
            done = True
            info['TimeLimit.truncated'] = True
        return observation, reward, done, info

    def reset(self, **kwargs):
        self._elapsed_steps = 0
        return self.env.reset(**kwargs)

In [ ]:
# Modify functions for Gif

def make_atari_2_ (env_id, max_episode_steps=None):
    env = gym.make(env_id)
    assert 'NoFrameskip' in env.spec.id
    env = NoopResetEnv(env, noop_max=30)
    env = MaxAndSkipEnv(env, skip=4)
    if max_episode_steps is not None:
        env = TimeLimit(env, max_episode_steps=max_episode_steps)
    return env

def wrap_deepmind_2_ (env, episode_life=True, clip_rewards=True, frame_stack=False, scale=False):
    """Configure environment for DeepMind-style Atari.
    """
    if episode_life:
        env = EpisodicLifeEnv(env)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    #env = WarpFrame(env)
    if scale:
        env = ScaledFloatFrame(env)
    if clip_rewards:
        env = ClipRewardEnv(env)
    if frame_stack:
        env = FrameStack(env, 4)
    return env

In [ ]:
# Use the Baseline Atari environment because of Deepmind helper functions
env = make_atari("BreakoutNoFrameskip-v4")

# Warp the frames, grey scale, stake four frame and scale to smaller ratio
env = wrap_deepmind(env, frame_stack=True, scale=True)
env.seed(7)


env2 = make_atari_2_("BreakoutNoFrameskip-v4")
env2 = wrap_deepmind_2_(env2, frame_stack=True, scale=True)
env2.seed(7)

frames = []
frames2 = []

model_learner.eval()
with torch.no_grad():

  for _ in range(5):
      t = 0
      reward_sum = 0.0
      reward_sum2 = 0.0

      state = env.reset()
      state2 = env2.reset()
      frames.append(state)
      frames2.append(state2[:,:,0:3])
      state = torch.tensor(np.transpose((np.array(state) * 255), (2, 0, 1)), dtype=torch.uint8).to(device)


      while True:
          t += 1 
          # ----------------------------------------------------------------------------
          # choose action (greedy)
          # ----------------------------------------------------------------------------
          model_learner.eval()
          with torch.no_grad():
            x = state.to(torch.float32).unsqueeze(0) / 255.0
            outputs = model_learner(x)
          action = np.argmax(outputs[0].tolist())     
          
          # ----------------------------------------------------------------------------
          # do the step 
          # ----------------------------------------------------------------------------
          next_state, reward, done, info = env.step(action)
          next_state2, reward2, done2, info2 = env2.step(action)

          frames.append(next_state)
          frames2.append(next_state2[:,:,0:3])
          next_state = torch.tensor(np.transpose((np.array(next_state) * 255), (2, 0, 1)), dtype=torch.uint8).to(device)
          reward_sum += reward
          reward_sum2 += reward2

          # ----------------------------------------------------------------------------
          # go to next_state 
          # ----------------------------------------------------------------------------
          state = next_state         
          
          if done:
              print(f"reward_sum -> {reward_sum} | reward_sum2 -> {reward_sum2} | num_step -> {t}")  
              break

env.close()

In [ ]:
# save_frames_as_gif(frames, filename='result.gif')
save_frames_as_gif(frames2, filename='result.gif')

**Your link: ```TODO ```** (e.g., https://drive.google.com/file/d/1WsT1_P5i5mXOSlzXpj-6cmGk9PIoIbWo/view?usp=sharing)

---

# Give Us Feedback

In [ ]:
from ipywidgets import Label, IntSlider, Textarea, Button, Layout, HBox, VBox
from IPython.display import clear_output

IntSlider_list = {i:IntSlider(value=3, min=1, max=5) for i in range(6)}
H1 = HBox([Label("How was the homework?")])
H2 = HBox([Label("Please rate the homework from 1 to 5 (lowest to highest).")])
H3 = HBox([Label("The assignment was instructive.", layout=Layout(width="22%")), IntSlider_list[0]])
H4 = HBox([Label("It was not time consuming.", layout=Layout(width="22%")), IntSlider_list[1]])
H5 = HBox([Label("The questions were clear.", layout=Layout(width="22%")), IntSlider_list[2]])
H6 = HBox([Label("The environment was interesting.", layout=Layout(width="22%")), IntSlider_list[3]])
H7 = HBox([Label("The homework was well organized.", layout=Layout(width="22%")), IntSlider_list[4]])
H8 = HBox([Label("Finally, you had a good vibe!", layout=Layout(width="22%")), IntSlider_list[5]])
textarea = Textarea(value='', placeholder='Any comments or suggestions', description='', disabled=False)
button = Button(description='Submit', disabled=False, button_style='', tooltip='Click me', icon='paper-plane')
H9 = HBox([textarea])
H10 = HBox([button])

def ff(button):
    button.button_style = 'success'
    score = sum([IntSlider_list[i].value for i in IntSlider_list]) / 6
    x = round(score*4)/4
    y = int(x) * '🌕' + ((x-int(x))==0 and not x==5) * '🌑' + ((x-int(x))==0.25) * '🌘' + ((x-int(x))==0.5) * '🌗' + ((x-int(x))==0.75) * '🌖' + (4-int(x)) * '🌑'
    clear_output(wait=True)
    display(VBox([H1, H2, H3, H4, H5, H6, H7, H8, H9, H10]))
    print("Thank you for your feedback! 😊")
    print(f"Score: {y}")
    print("\nFeedback:",*[IntSlider_list[i].value for i in IntSlider_list])
    print(textarea.value)

button.on_click(ff)

VBox([H1, H2, H3, H4, H5, H6, H7, H8, H9, H10])

In [1]:
# To export jupyter notebook to html, save your notebook and run this cell
!jupyter nbconvert --to html CHW3.ipynb

[NbConvertApp] Converting notebook CHW3.ipynb to html
[NbConvertApp] Writing 789879 bytes to CHW3.html
